In [1]:
import header
from v0.models import Image, Content
from v0.index import image_index
import pandas as pd
import random
import imquality.brisque as brisque
from PIL import Image
import requests
import urllib.parse

DJANGO SETTINGS IN DEBUG
[sentence_transformers.SentenceTransformer] 2022-06-02 15:21:06,558 INFO MainThread [SentenceTransformer:__init__:60] | Load pretrained SentenceTransformer: all-mpnet-base-v2
[sentence_transformers.SentenceTransformer] 2022-06-02 15:21:14,808 INFO MainThread [SentenceTransformer:__init__:97] | Use pytorch device: cpu
[faiss.loader] 2022-06-02 15:21:14,892 INFO MainThread [loader:<module>:64] | Loading faiss.
[faiss.loader] 2022-06-02 15:21:14,911 INFO MainThread [loader:<module>:66] | Successfully loaded faiss.
[VectorIndex_Image] 2022-06-02 15:21:16,983 INFO MainThread [index:_generate_index:46] | Generated index with a total of 2516 vectors in 2.0708s


In [2]:
# get 100 random content
random_pks = random.sample(list(Content.objects.exclude(thumbnail='https://miro.medium.com/').values_list('pk', flat=True)), 100)
contents = Content.objects.filter(pk__in=random_pks)


In [3]:
rows = []
url = f'https://api.apilayer.com/image_quality/url?url='
headers = {'apikey': 'myusi6zK4HKqiSUllMVp152hqq0al8C1'}
MAX_SIZE = (500, 500)

for content in contents:
    try:
        content: Content
        content_row = {'article_url': content.url, 'original_img': content.thumbnail}

        # get new image via image_index
        results = image_index.query(content.embedding_all_mpnet_base_v2, k=1)
        new_img = results[0][0]
        content_row['new_img'] = new_img.url
        content_row['new_old_semantic_similarity'] = results[1][0][1]

        # apply brisque to both new and old images
        # r = requests.get(url + urllib.parse.quote(content.thumbnail), headers=headers)
        # content_row['original_image_brisque'] = r.json()['score']
        ardvark = Image.open(requests.get(content.thumbnail, timeout=3, stream=True).raw)
        ardvark.thumbnail(MAX_SIZE)
        content_row['original_img_brisque'] = brisque.score(ardvark)

        # r = requests.get(url + urllib.parse.quote(new_img.url), headers=headers)
        # content_row['new_img_brisque'] = r.json()['score']
        ardvark = Image.open(requests.get(new_img.url, timeout=3, stream=True).raw)
        ardvark.thumbnail(MAX_SIZE)
        content_row['new_img_brisque'] = brisque.score(ardvark)

        rows.append(content_row)
    except Exception as e:
        print(e)

df = pd.DataFrame(rows, columns=['article_url', 'original_img', 'original_img_brisque', 'new_img', 'new_img_brisque', 'new_old_semantic_similarity'])
df.to_csv('image_experiment.csv', index=False)

[VectorIndex_Image] 2022-06-02 15:21:43,207 INFO MainThread [index:query:116] | Searched index and got 1 vectors in 0.0062s
[VectorIndex_Image] 2022-06-02 15:22:04,193 INFO MainThread [index:query:116] | Searched index and got 1 vectors in 0.001s
[VectorIndex_Image] 2022-06-02 15:22:07,844 INFO MainThread [index:query:116] | Searched index and got 1 vectors in 0.0015s
[VectorIndex_Image] 2022-06-02 15:22:09,553 INFO MainThread [index:query:116] | Searched index and got 1 vectors in 0.0017s
[VectorIndex_Image] 2022-06-02 15:22:13,256 INFO MainThread [index:query:116] | Searched index and got 1 vectors in 0.0006s
[VectorIndex_Image] 2022-06-02 15:22:15,212 INFO MainThread [index:query:116] | Searched index and got 1 vectors in 0.0007s
[VectorIndex_Image] 2022-06-02 15:22:21,456 INFO MainThread [index:query:116] | Searched index and got 1 vectors in 0.0006s
[VectorIndex_Image] 2022-06-02 15:22:23,659 INFO MainThread [index:query:116] | Searched index and got 1 vectors in 0.0018s
[VectorIn